<a href="https://colab.research.google.com/github/poojithamoganti/Semantic_search/blob/main/Semantic_Search_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Semantic Search using transformers and Multi news database

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,util
import torch

In [ ]:
dataset = load_dataset("multi_news" , split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
df = dataset.to_pandas().sample(2000, random_state = 42)

In [ ]:
print(df)

                                               document  \
4830  Tweet with a location \n \n You can add locati...   
1255  CNN host Piers Morgan just called to discuss h...   
80    White House communications director Anthony Sc...   
3044  CLOSE Scientists say they've found archaeologi...   
4486  Click image above to view graphic \n \n Althou...   
...                                                 ...   
2157  On Thursday afternoon, President-elect Donald ...   
3615  Donald Trump said Sunday that in the wake of t...   
2751  Nashua police believe body found is that of mi...   
622   The public school systems in New York and Los ...   
2252  For the past 3 years Lauriann has been cared f...   

                                                summary  
4830  – Denis Finley has taken to Twitter to call Po...  
1255  – CNN's Piers Morgan thinks gun-rights propone...  
80    – New White House communications director Anth...  
3044  – Scientists say they have the first physical ...  
4

We will use embeddings in order to pair queries and documents. Setence Transformer is used. Sentence transformer gives a single representation for an entire sentence

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

For Each news arcticle we have a single vector representation capturing its meaning in a compact form.This makes it easy to find most relevant news arcticles.

In [ ]:
passage_embeddings = list(model.encode(df['summary'].tolist(), show_progress_bar=True))

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
passage_embeddings[0].shape

(384,)

Query to find few relevant articles regarding Technology and AI

In order to compare this query with news dataset, we need to transform query into an embedding using the same model used for summeries.

We use cosine similarity function to calculate the similarity between two embedding vectors

Finding top 3 indices that correspond to the most similar passages using Torch top K function

Retreiving summaries with the highest scores

Making it a function to retreive information for other queries

In [ ]:
def find_relevant_news(query):
  query_embeddings = model.encode(query)
  query_embeddings.shape

  similarities = util.cos_sim(query_embeddings, passage_embeddings)
  similarities.shape

  top_indices = torch.topk(similarities.flatten(), 3).indices

  top_relevant = [df.iloc[x.item()]['summary'][:200] + "..." for x in top_indices]

  return top_relevant

In [ ]:
find_relevant_news("Movie")

['– "Chilling," "unsettling," and "manipulative" is what critics are saying about Gone Girl, starring Ben Affleck and Rosamund Pike as Nick and Amy, a husband and wife trying to make it in a small town ...',
 '– Repo Men, a sci-fi thriller about men who repossess organs, is probably supposed to be biting commentary on divisive issues like health care, but critics are united in shouting it down: “Let the 201...',
 "– Kathryn Bigelow's Detroit tells the story of a black singer who takes refuge in a dingy motel to escape the 1967 Motor City riots. Based on real events, the film reunites Bigelow with screenwriter M..."]

In [ ]:
find_relevant_news("Natural Disasters")

['– The tsunami that killed hundreds, possibly thousands of people after an earthquake in Indonesia on Friday was much bigger and more devastating than would normally be expected after that kind of quak...',
 '– A sad milestone out of Japan: Two weeks after the quake struck, its official death toll has broken the 10,000 mark—and that number is still on the rise, with more than 17,400 missing. Police estimat...',
 '– When you live near a major dam, the last thing you want to hear is that the integrity of it has been "compromised" by landslides. But that\'s exactly what residents living below North Carolina\'s Lake...']

In [ ]:
find_relevant_news("Crime")

['– San Francisco-area cops have arrested three more young men in the savage gang rape, beating, and robbery of a 15-year-old girl outside her homecoming dance while dozens more watched. Five youths are...',
 '– "I\'m a born burglar," says Octave Durham, per the New York Times, which looks at a new documentary about Durham\'s notorious theft of two Vincent Van Gogh paintings. Set to be broadcast Tuesday on Du...',
 '– The number of murders in the US rose 11% last year from 2014, while the number of violent crimes saw a modest increase of 4% after two years of decline, per an FBI press release. Rapes rose 6.3%, ag...']